In [ ]:
# default_exp util

In [ ]:
#hide
from nbdev.showdoc import *

# ProofZero Utility SDK

This

## Exported types

In [ ]:
#export
from typing import NewType, Any
Hash = NewType('Hash', str)

## Hash functions

In [ ]:
#export
def noop(plainvalue: Any) -> Hash:
    """
    A no-op hash function (ie, passthrough, no encryption applied). Useful for testing.
    """
    return repr(plainvalue)

In [ ]:
#export
import pandas as pd
def sha2(plainvalue: Any) -> Hash:
    """
    A cross-platform implementation wrapper around SHA2. Generates consistent hashes across different hardware and interpreter platforms.
    """
    # Infer type using Python built-in so we can set formatting string.
    type_hint = type(plainvalue)

    # Prepend '!' to force system-indepent memory formatting.
    # See https://docs.python.org/3/library/struct.html#struct-format-strings
    format_code = "!"

    # Select based on type_hint.
    # See https://docs.python.org/3/library/struct.html#format-characters
    if type_hint is int:
        format_code = "".join([format_code, "i"])
    elif type_hint is float:
        format_code = "".join([format_code, "d"])
    elif type_hint is bool:
        format_code = "".join([format_code, "?"])
    elif type_hint is str:
        # Decode the string into an immutable byte buffer (see below).
        value = bytes(value, encoding)
        format_code = "".join([format_code, str(len(value)), "s"])
    else:
        # Best-efforts conversion of the value into a string representation.
        value = repr(value)

        # Now treat it as a string (see above).
        value = bytes(value, encoding)
        format_code = "".join([format_code, str(len(value)), "s"])

    # Pack the value into a struct so we can get its memory representation.
    # The format_code having a prepended '!' ensures platform independence.
    value_buf = struct.pack(format_code, value)

    # Hash the buffer and return the hex string.
    return Hash(hashlib.sha256(value_buf).hexdigest())

## Parsing functions